## Project 1: Language Modeling

In this project, you will implement several different types of language models for text.  We'll start with n-gram models, then move on to neural n-gram and LSTM language models.

Warning: Do not start this project the day before it is due!  Some parts require 20 minutes or more to run, so debugging and tuning can take a significant amount of time.

Our dataset for this project will be the WikiText2 language modeling dataset.  This dataset comes with some of the basic preprocessing done for us, such as tokenization and rare word filtering (using the `<unk>` token).
Therefore, we can assume that all word types in the test set also appear at least once in the training set.
We'll also use the `torchtext` library to help with some of the data preprocessing, such as converting tokens into id numbers.

In [2]:
# This block handles some basic setup and data loading.  
# You shouldn't need to edit this, but if you want to 
# import other standard python packages, that is fine.

# imports
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import os

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

# download and load the data
text_field = torchtext.data.Field()
datasets = torchtext.datasets.WikiText2.splits(root='.', text_field=text_field)
train_dataset, validation_dataset, test_dataset = datasets
text_field.build_vocab(train_dataset)
vocab = text_field.vocab
vocab_size = len(vocab)

train_text = train_dataset.examples[0].text # a list of tokens (strings)
validation_text = validation_dataset.examples[0].text

print(validation_text[:30])

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:00<00:00, 6.53MB/s]


extracting
['<eos>', '=', 'Homarus', 'gammarus', '=', '<eos>', '<eos>', 'Homarus', 'gammarus', ',', 'known', 'as', 'the', 'European', 'lobster', 'or', 'common', 'lobster', ',', 'is', 'a', 'species', 'of', '<unk>', 'lobster', 'from', 'the', 'eastern', 'Atlantic', 'Ocean']


We've implemented a unigram model here as a demonstration.

In [3]:
class UnigramModel:
    def __init__(self, train_text):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)

    def probability(self, word):
        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""
        return [self.probability(word) for word in vocab.itos]

    def perplexity(self, full_text):
        """Return the perplexity of the model on a text as a float.
        
        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter 
            # as long as the log and exp use the same base.
            log_probabilities.append(math.log(self.probability(word), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(train_text)
print('unigram validation perplexity:', 
      unigram_demonstration_model.perplexity(validation_text))

def check_validity(model):
    """Performs several sanity checks on your model:
    1) That next_word_probabilities returns a valid distribution
    2) That perplexity matches a perplexity calculated from next_word_probabilities

    Although it is possible to calculate perplexity from next_word_probabilities, 
    it is still good to have a separate more efficient method that only computes 
    the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = validation_text[:i]
        probs = model.next_word_probabilities(prefix)
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab.stoi[validation_text[i]]
        selected_prob = probs[word_id]
        log_probabilities.append(math.log(selected_prob))

    perplexity = math.exp(-np.mean(log_probabilities))
    your_perplexity = model.perplexity(validation_text[:10])
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


check_validity(unigram_demonstration_model)

unigram validation perplexity: 965.0860734119312


To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [4]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(vocab.itos, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

print(generate_text(unigram_demonstration_model))

<eos> <eos> allowed ASCAP its ) <eos> regularly Turner whether . very turned to Eurasia work ' , living 1 . on


In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this recent paper: https://arxiv.org/pdf/1904.09751.pdf.

You will need to submit some outputs from the models you implement for us to grade.  The following function will be used to generate the required output files.

In [5]:
!wget https://cal-cs288.github.io/sp20/project_files/proj_1/eval_prefixes.txt
!wget https://cal-cs288.github.io/sp20/project_files/proj_1/eval_output_vocab.txt

def save_truncated_distribution(model, filename):
    """Generate a file of truncated distributions.
    
    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    with open('eval_output_vocab.txt', 'r') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    eval_vocab_ids = [vocab.stoi[s] for s in eval_vocab]

    all_selected_probabilities = []
    with open('eval_prefixes.txt', 'r') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        for line in tqdm.tqdm_notebook(lines, leave=False):
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

save_truncated_distribution(unigram_demonstration_model, 
                            'unigram_demonstration_predictions.npy')

--2020-02-21 05:19:25--  https://cal-cs288.github.io/sp20/project_files/proj_1/eval_prefixes.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105976 (103K) [text/plain]
Saving to: ‘eval_prefixes.txt’

eval_prefixes.txt   100%[===================>] 103.49K  --.-KB/s    in 0.01s   

2020-02-21 05:19:25 (8.75 MB/s) - ‘eval_prefixes.txt’ saved [105976/105976]

--2020-02-21 05:19:27--  https://cal-cs288.github.io/sp20/project_files/proj_1/eval_output_vocab.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3802 (3.7K) [text/plain]
Saving to: ‘eval_output_vocab.txt’

eval_output_

saved unigram_demonstration_predictions.npy


### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use add-alpha smoothing to make sure no output word has probability 0.

$$P(w_2|w_1)=\frac{C(w_1,w_2)+\alpha}{C(w_1)+N\alpha}$$

where $N$ is the vocab size.  An alpha value around `3e-3`  should work.  Later, we'll replace this smoothing with model backoff.

One edge case you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You can handle this however you like as long as you produce a valid probability distribution, but just using a uniform distribution over the vocabulary is reasonable for the purposes of this project.

A properly implemented bi-gram model should get a perplexity below 510 on the validation set.

Note: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass.

In [6]:
class NGramModel:
    def __init__(self, train_text, n=2, alpha=3e-3):
        # get counts and perform any other setup
        self.n = n
        self.smoothing = alpha

        # YOUR CODE HERE

        self.total = len(train_text)
        self.single_counts = Counter(train_text)

        self.n_1_gram_counts = Counter( [ tuple(train_text[i:i + n - 1])
                                          for i in range(self.total - n + 2)
                                        ]
                                      )
        self.n_gram_counts = Counter( [ tuple(train_text[i:i + n])
                                        for i in range(self.total - n + 1)
                                      ]
                                    )

    def n_gram_probability(self, n_gram):
        """Return the probability of the last word in an n-gram.
        
        n_gram -- a list of string tokens
        returns the conditional probability of the last token given the rest.
        """
        assert len(n_gram) == self.n

        # YOUR CODE HERE

        return (self.n_gram_counts[tuple(n_gram)] + self.smoothing) / \
               (self.n_1_gram_counts[tuple(n_gram[:-1])] + vocab_size * self.smoothing)

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""

        # YOUR CODE HERE
        # use your function n_gram_probability
        # vocab.itos contains a list of words to return probabilities for

        if len(text_prefix) < self.n - 1:
          # return [1 / vocab_size] * vocab_size    # Use a uniform distribution over the vocabulary
          return [self.single_counts[word] / self.total for word in vocab.itos]

        prefix_len = len(text_prefix)
        all_probabilities = [ self.n_gram_probability( text_prefix[prefix_len - (self.n - 1) : prefix_len] + [word] )
                              for word in vocab.itos
                            ]
        return all_probabilities

    def perplexity(self, full_text):
        """ full_text is a list of string tokens
        return perplexity as a float """

        # YOUR CODE HERE
        # use your function n_gram_probability
        # This method should differ a bit from the example unigram model because 
        # the first n-1 words of full_text must be handled as a special case.

        log_probabilities = []
        n_gram = []
        for i, word in enumerate(full_text):
          if i < self.n - 1:
            n_gram.append(word)
            # log_probabilities.append( math.log(1 / vocab_size, 2) )    # Use a uniform distribution over the vocabulary
            log_probabilities.append( math.log(self.single_counts[word] / self.total, 2) )
          else:
            n_gram.append(word)
            log_probabilities.append( math.log(self.n_gram_probability(n_gram), 2) )
            n_gram.pop(0)

        return 2 ** -np.mean(log_probabilities)

unigram_model = NGramModel(train_text, 1)
check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(validation_text)) # this should be the almost the same as our unigram model perplexity above

bigram_model = NGramModel(train_text, n=2)
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(validation_text))

trigram_model = NGramModel(train_text, n=3)
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(validation_text)) # this won't do very well...

save_truncated_distribution(bigram_model, 'bigram_predictions.npy') # this might take a few minutes

unigram validation perplexity: 965.0918307092597
bigram validation perplexity: 504.4148539982192
trigram validation perplexity: 2965.494861349281


saved bigram_predictions.npy


Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

In the block below, please report your bigram validation perplexity.  (We will use this to help us calibrate our scoring on the test set.)

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Bigram validation perplexity: ***504.4148539982192***

We can also generate samples from the model to get an idea of how it is doing.

In [7]:
print(generate_text(bigram_model))

<eos> <eos> A Star Teams 020 irrelevant unique to the Oricon staunch aging Nat eminently reside Mildred CMU HD launching Niagara animal


This basic model works okay for bigrams, but a better strategy (especially for higher-order models) is to use backoff.  Implement backoff with absolute discounting.
$$P\left(w_i|w_{i-n+1}^{i-1}\right)=\frac{max\left\{C(w_{i-n+1}^i)-\delta,0\right\}}{\sum_{w_i} C(w_{i-n+1}^i)} + \alpha(w_{i-n+1}^{i-1}) P(w_i|w_{i-n+2}^{i-1})$$

$$\alpha\left(w_{i-n+1}^{i-1}\right)=\frac{\delta N_{1+}(w_{i-n+1}^{i-1})}{{\sum_{w_i} C(w_{i-n+1}^i)}}$$
where $N_{1+}$ is the number of words that appear after the previous $n-1$ words (the number of times the max will select something other than 0 in the first equation).  If $\sum_{w_i} C(w_{i-n+1}^i)=0$, use the lower order model probability directly (the above equations would have a division by 0).

We found a discount $\delta$ of 0.9 to work well based on validation performance.  A trigram model with this discount value should get a validation perplexity below 275.

In [8]:
from collections import defaultdict

class DiscountBackoffModel(NGramModel):
    def __init__(self, train_text, lower_order_model, n=2, delta=0.9):
        super().__init__(train_text, n=n)
        self.lower_order_model = lower_order_model
        self.discount = delta

        # YOUR CODE HERE

        self.n_1_gram_word_types = defaultdict(set)

        for i in range(self.total - n + 1):
          key = tuple( train_text[i : i + n - 1] )
          self.n_1_gram_word_types[key].add( train_text[i + n - 1] )

    def n_gram_probability(self, n_gram):
        assert len(n_gram) == self.n

        # YOUR CODE HERE
        # back off to the lower_order model with n'=n-1 using its n_gram_probability function

        n_gram_key, n_1_gram_key = tuple(n_gram), tuple(n_gram[:-1])
        lower_model_n_gram_prob = self.lower_order_model.n_gram_probability(n_gram[1:])

        if self.n_1_gram_counts[n_1_gram_key] == 0:
          return lower_model_n_gram_prob

        discounted_prob = max(self.n_gram_counts[n_gram_key] - self.discount, 0) / \
                          self.n_1_gram_counts[n_1_gram_key]

        alpha = self.discount * len(self.n_1_gram_word_types[n_1_gram_key]) / \
                self.n_1_gram_counts[n_1_gram_key]

        return discounted_prob + alpha * lower_model_n_gram_prob

bigram_backoff_model = DiscountBackoffModel(train_text, unigram_model, 2)
trigram_backoff_model = DiscountBackoffModel(train_text, bigram_backoff_model, 3)
check_validity(trigram_backoff_model)
print('trigram backoff validation perplexity:', trigram_backoff_model.perplexity(validation_text))

trigram backoff validation perplexity: 271.10625333491447


Now, implement Kneser-Ney to replace the unigram base model.
$$P(w)\propto |\{w':c(w',w) > 0\}|$$
A Kneser-Ney trigram model should get a validation perplexity below 260.

In [9]:
class KneserNeyBaseModel(NGramModel):
    def __init__(self, train_text):
        super().__init__(train_text, n=1)

        # YOUR CODE HERE

        self.unigram_word_types = defaultdict(set)

        for i in range(self.total - 1):
          self.unigram_word_types[train_text[i + 1]].add(train_text[i])

        self.bigram_types = set([ tuple(train_text[i : i + 2]) for i in range(self.total - 1) ])

    def n_gram_probability(self, n_gram):
        assert len(n_gram) == 1

        # YOUR CODE HERE

        return len(self.unigram_word_types[n_gram[0]]) / len(self.bigram_types)

kn_base = KneserNeyBaseModel(train_text)
check_validity(kn_base)
bigram_kn_backoff_model = DiscountBackoffModel(train_text, kn_base, 2)
trigram_kn_backoff_model = DiscountBackoffModel(train_text, bigram_kn_backoff_model, 3)
print('trigram Kneser-Ney backoff validation perplexity:', trigram_kn_backoff_model.perplexity(validation_text))

save_truncated_distribution(trigram_kn_backoff_model, 'trigram_kn_predictions.npy') # this might take a few minutes

trigram Kneser-Ney backoff validation perplexity: 256.59871461524637


saved trigram_kn_predictions.npy


In [10]:
print(generate_text(trigram_kn_backoff_model))
print(generate_text(trigram_kn_backoff_model, prefix=['What','about']))

<eos> <eos> = = = = = = <eos> <eos> = = <eos> <eos> " Lisa 's First World War Z 's
What about 8 mm ) fell separate the <unk> Driver represent a He may have a central ore Tudor conquest corn crakes


Fill in your trigram backoff perplexities with and without Kneser Ney.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Trigram backoff validation perplexity: ***271.10625333491447***

Trigram backoff with Kneser Ney perplexity: ***256.59871461524637***

### Neural N-gram Model

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` words and outputs a distribution over the next word.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using PyTorch's data loaders (https://pytorch.org/docs/stable/data.html)

A model with the following architecture and hyperparameters should reach a validation perplexity below 226.
* embed the words with dimension 128, then flatten into a single embedding for $n-1$ words (with size $(n-1)*128$)
* run 2 hidden layers with 1024 hidden units, then project down to size 128 before the final layer
* use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); you can do this by creating the output layer with `nn.Linear`, then using `F.embedding` with the linear layer's `.weight` to embed the input
* rectified linear activation (ReLU) and dropout 0.1 on each hidden layer
* train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)
* do early stopping based on validation set perplexity (see Project 0)


We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above.  A proper implementation with these should be enough to receive full credit on the assignment, though.

In [0]:
def ids(tokens):
    return [vocab.stoi[t] for t in tokens]

assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

In [14]:
class NeuralNgramDataset(torch.utils.data.Dataset):
    def __init__(self, text_token_ids, n):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i):
        if i < self.n-1:
            prev_token_ids = [vocab.stoi['<eos>']] * (self.n-i-1) + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i-self.n+1:i]

        assert len(prev_token_ids) == self.n-1

        x = torch.tensor(prev_token_ids)
        y = torch.tensor(self.text_token_ids[i])
        return x, y

class NeuralNGramNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self, n):
        super().__init__()
        self.n = n

        # YOUR CODE HERE

        embedding_dim = 128
        hidden_size_1 = 1024
        hidden_size_2 = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.module = nn.Sequential(nn.Flatten(),
                               nn.Linear((self.n - 1) * embedding_dim, hidden_size_1),
                               nn.ReLU(),
                               nn.Dropout(p=0.1),
                               nn.Linear(hidden_size_1, hidden_size_1),
                               nn.ReLU(),
                               nn.Dropout(p=0.1),
                               nn.Linear(hidden_size_1, hidden_size_2),
                              )
        self.last_linear = nn.Linear(hidden_size_2, vocab_size)
        self.embedding.weight = self.last_linear.weight

    def forward(self, x):
        # x is a tensor of inputs with shape (batch, n-1)
        # this function returns a tensor of log probabilities with shape (batch, vocab_size)

        # YOUR CODE HERE

        # x = F.embedding(x, weight=self.last_linear.weight)    # Alternatively, share weights using embedding function
        x = self.embedding(x)
        x = self.module(x)
        x = self.last_linear(x)
        outputs = x
        return outputs

class NeuralNGramModel:
    # a class that wraps NeuralNGramNetwork to handle training and evaluation
    # it's ok if this doesn't work for unigram modeling
    def __init__(self, n):
        self.n = n
        self.network = NeuralNGramNetwork(n).cuda()

    def train(self):
        dataset = NeuralNgramDataset(ids(train_text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)
        # iterating over train_loader with a for loop will return a 2-tuple of batched tensors
        # the first tensor will be previous token ids with size (batch, n-1),
        # and the second will be the current token id with size (batch, )
        # you will need to move these tensors to GPU, e.g. by using the Tensor.cuda() function.

        # this will take some time to run; use tqdm.tqdm_notebook to get a progress bar 
        # (see Project 0 for example)

        # YOUR CODE HERE

        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.network.parameters())
        best_perplexity = 99999999

        if not os.path.exists("models"):
          os.makedirs("models")

        for epoch in range(10):
          print("Epoch", epoch)

          for batch in tqdm.tqdm_notebook(train_loader, leave=False):
            x, y_true = batch
            assert self.network.training, "Not in training mode. Switch to it using .train()"
            optimizer.zero_grad()
            x, y_true = x.cuda(), y_true.cuda()
            outputs = self.network(x)
            loss = criterion(outputs, y_true)
            loss.backward()
            optimizer.step()

          validation_perplexity = self.perplexity(validation_text)
          print("Validation perplexity", validation_perplexity)

          if validation_perplexity < best_perplexity:
            best_perplexity = validation_perplexity
            torch.save(self.network.state_dict(), "models/neural_n_gram_model")

        self.network.load_state_dict(torch.load("models/neural_n_gram_model"))

    def next_word_probabilities(self, text_prefix):
        # YOUR CODE HERE
        # don't forget self.network.eval()
        # you will need to convert text_prefix from strings to numbers with the `ids` function
        # if your `perplexity` function below is based on a NeuralNgramDataset DataLoader, you will need to use the same strategy for prefixes with less than n-1 tokens to pass the validity check
        #   the data loader appends extra "<eos>" (end of sentence) tokens to the start of the input so there are always enough to run the network

        self.network.eval()

        text_prefix_len = len(text_prefix)
        if text_prefix_len < self.n - 1:
          text_prefix = ["<eos>"] * (self.n - 1 - text_prefix_len) + text_prefix

        text_prefix_ids = torch.tensor( ids(text_prefix) ).cuda()
        outputs = self.network( text_prefix_ids[-(self.n - 1):].unsqueeze(0) )
        probabilities = F.softmax(outputs[0], dim=0)

        self.network.train()
        return probabilities

    def perplexity(self, text):
        # you may want to use a DataLoader here with a NeuralNgramDataset
        # don't forget self.network.eval()

        # YOUR CODE HERE

        self.network.eval()

        text_ids = ids(text)
        dataset = NeuralNgramDataset(text_ids, self.n)
        data_loader = torch.utils.data.DataLoader(dataset)

        log_probabilities = []
        for x, y_true in data_loader:
          x, y_true = x.cuda(), y_true.cuda()
          outputs = self.network(x)
          probabilities = F.softmax(outputs[0], dim=0)
          probability = probabilities[y_true[0]]
          log_probabilities.append( math.log(probability, 2) )

        self.network.train()
        return 2 ** -np.mean(log_probabilities)


neural_trigram_model = NeuralNGramModel(3)
check_validity(neural_trigram_model)
neural_trigram_model.train()
print('neural trigram validation perplexity:', neural_trigram_model.perplexity(validation_text))

save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy')

Epoch 0


Validation perplexity 236.49561639244746
Epoch 1


Validation perplexity 212.6035183684932
Epoch 2


Validation perplexity 206.44909828516535
Epoch 3


Validation perplexity 208.38725307415257
Epoch 4


...

Validation perplexity 218.61041562280082
Epoch 7


Validation perplexity 221.10486224826735
Epoch 8


Validation perplexity 223.83545890281187
Epoch 9


Validation perplexity 227.43849735634717
neural trigram validation perplexity: 206.44909828516535


saved neural_trigram_predictions.npy


Fill in your neural trigram perplexity.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Neural trigram validation perplexity: ***206.44909828516535***

### LSTM Model

For this stage of the project, you will implement an LSTM language model.

For recurrent language modeling, the data batching strategy is a bit different from what is used in some other tasks.  Sentences are concatenated together so that one sentence starts right after the other, and an unfinished sentence will be continued in the next batch.  We'll use the `torchtext` library to manage this batching for you.  To properly deal with this input format, you should save the last state of the LSTM from a batch to feed in as the first state of the next batch.  When you save state across different batches, you should call `.detach()` on the state tensors before the next batch to tell PyTorch not to backpropagate gradients through the state into the batch you have already finished (which will cause a runtime error).

We expect your model to reach a validation perplexity below 130.  The following architecture and hyperparameters should be sufficient to get there.
* 3 LSTM layers with 512 units
* dropout of 0.5 after each LSTM layer
* instead of projecting directly from the last LSTM output to the vocabulary size for softmax, project down to a smaller size first (e.g. 512->128->vocab_size)
* use the same weights for the embedding layer and the pre-softmax layer; dimension 128
* train with Adam (using default learning rates) for at least 20 epochs


In [18]:
class LSTMNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self):
        super().__init__()

        # YOUR CODE HERE

        embedding_dim = 512
        self.input_size = embedding_dim
        self.hidden_size = 512
        self.num_layers = 3
        hidden_size_2 = embedding_dim

        # self.embedding = nn.Embedding(vocab_size, embedding_dim)    # Alternatively, create embedding layer tying weights to those of output layer
        self.lstm = nn.LSTM(self.input_size,
                            self.hidden_size,
                            num_layers=self.num_layers,
                            dropout=0.5
                           )
        self.dropout = nn.Dropout(p=0.5)
        self.shrink_linear = nn.Linear(self.hidden_size, hidden_size_2)
        self.output_linear = nn.Linear(hidden_size_2, vocab_size)
        # self.embedding.weight = self.output_linear.weight    # Tying to weights of output layer

    def forward(self, x, state):
        """Compute the output of the network.
        
        Note: In the Pytorch LSTM tutorial, the state variable is named "hidden":
        https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

        The torch.nn.LSTM documentation is quite helpful:
        https://pytorch.org/docs/stable/nn.html#lstm
    
        x - a tensor of int64 inputs with shape (seq_len, batch)
        state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
                representing the hidden state and cell state of the of the LSTM.
        returns a tuple with two elements:
          - a tensor of log probabilities with shape (seq_len, batch, vocab_size)
          - a state tuple returned by applying the LSTM.
        """

        # Note that the nn.LSTM module expects inputs with the sequence 
        # dimension before the batch by default.
        # In this case the dimensions are already in the right order, 
        # but watch out for this since sometimes people put the batch first

        # YOUR CODE HERE

        x = F.embedding(x, weight=self.output_linear.weight)
        # x = self.embedding(x)
        x_out, state = self.lstm(x, state)
        x_out = self.dropout(x_out)
        x_out = self.shrink_linear(x_out)
        x_out = self.output_linear(x_out)
        outputs = F.log_softmax(x_out, dim=-1)
        return outputs, state

class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."

    def __init__(self):
        self.network = LSTMNetwork().cuda()

    def train(self):
        train_iterator = torchtext.data.BPTTIterator(train_dataset, batch_size=64, 
                                                     bptt_len=32, device='cuda')
        # Iterate over train_iterator with a for loop to get batches
        # each batch object has a .text and .target attribute with
        # token id tensors for the input and output respectively.

        # The initial state passed into the LSTM should be set to zero.

        # YOUR CODE HERE

        criterion = torch.nn.NLLLoss()
        optimizer = torch.optim.Adam(self.network.parameters())
        lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 12, 14, 16, 18], gamma=0.25)

        best_perplexity = 999999
        batch_size = 64

        if not os.path.exists("models"):
          os.makedirs("models")

        for epoch in range(20):
          print("Epoch", epoch)

          h_0 = torch.zeros(self.network.num_layers, batch_size, self.network.hidden_size).cuda()
          c_0 = torch.zeros(self.network.num_layers, batch_size, self.network.hidden_size).cuda()
          state_0 = (h_0, c_0)

          for batch in tqdm.tqdm_notebook(train_iterator, leave=False):
            x, y_true = batch.text, batch.target
            assert self.network.training, "Not in training mode. Switch to it using .train()"
            optimizer.zero_grad()
            outputs, (h_T, c_T) = self.network(x, state_0)
            loss = criterion(outputs.view(-1, vocab_size), y_true.flatten())
            loss.backward()
            optimizer.step()
            state_0 = (h_T.detach(), c_T.detach())

          validation_perplexity = self.dataset_perplexity(validation_dataset)
          print("Validation perplexity", validation_perplexity)

          if validation_perplexity < best_perplexity:
            best_perplexity = validation_perplexity
            torch.save(self.network.state_dict(), "models/lstm_n_gram_model")

          lr_scheduler.step()

        self.network.load_state_dict(torch.load("models/lstm_n_gram_model"))

    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."

        prefix_token_tensor = torch.tensor(ids(text_prefix), device='cuda').view(-1, 1)
        
        # YOUR CODE HERE

        self.network.eval()

        batch_size = 1
        h_0 = torch.zeros(self.network.num_layers, batch_size, self.network.hidden_size).cuda()
        c_0 = torch.zeros(self.network.num_layers, batch_size, self.network.hidden_size).cuda()
        state_0 = (h_0, c_0)

        with torch.no_grad():
          log_probabilities, _ = self.network(prefix_token_tensor, state_0)

        probabilities = torch.exp( log_probabilities.squeeze()[-1] ).tolist()

        self.network.train()
        return probabilities

    # @torch.no_grad()    # Alternatively, use decorator to stop taking grad
    def dataset_perplexity(self, torchtext_dataset):
        "Return perplexity as a float."
        # Your code should be very similar to next_word_probabilities, but
        # run in a loop over batches. Use torch.no_grad() for extra speed.

        iterator = torchtext.data.BPTTIterator(torchtext_dataset, batch_size=64, bptt_len=32, device='cuda')

        # YOUR CODE HERE

        self.network.eval()
        # torch.set_grad_enabled(False)    # Alternatively, set grad to False to stop taking grad
                                           # this works globally so remember to set it back to True
        batch_size = 64
        h_0 = torch.zeros(self.network.num_layers, batch_size, self.network.hidden_size).cuda()
        c_0 = torch.zeros(self.network.num_layers, batch_size, self.network.hidden_size).cuda()
        state_0 = (h_0, c_0)

        log_probabilities = []
        for batch in iterator:
          x, y_true = batch.text, batch.target

          with torch.no_grad():    # Disable grad for evaluating model to save from using memory and to speed up
            outputs, (h_T, c_T) = self.network(x, state_0)
            state_0 = (h_T.detach(), c_T.detach())

          selected_log_probabilities = outputs.gather(-1, y_true.unsqueeze(-1))
          selected_log_prob_list = selected_log_probabilities.flatten().tolist()
          log_probabilities.extend(selected_log_prob_list)

        # torch.set_grad_enabled(True)    # Set grad to True
        self.network.train()
        return np.exp(-np.mean(log_probabilities))

lstm_model = LSTMModel()
lstm_model.train()

print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy')

Epoch 0


Validation perplexity 257.16100202433637
Epoch 1


Validation perplexity 178.0136890113688
Epoch 2


Validation perplexity 150.2978843107591
Epoch 3


Validation perplexity 141.4494358412642
Epoch 4


...

Validation perplexity 109.88399888556614
Epoch 17


Validation perplexity 109.71943673697061
Epoch 18


Validation perplexity 109.60574038469902
Epoch 19


Validation perplexity 109.53816410072704
lstm validation perplexity: 109.53816410072704


saved lstm_predictions.npy


Now it's time for you to experiment.  Try to reach a validation perplexity below 120.

It is okay if the bulk of your improvements are due to hyperparameter tuning (such as changing number or sizes of layers), but implement at least one more substantial change to the model.  Here are some ideas (several of which come from https://arxiv.org/pdf/1708.02182.pdf):
* activation regularization - add a l2 regularization penalty on the activation of the LSTM output (standard l2 regularization is on the weights)
* weight-drop regularization - apply dropout to the weight matrices instead of activations
* learning rate scheduling - decrease the learning rate during training
* embedding dropout - zero out the entire embedding for a random set of words in the embedding matrix
* ensembling - average the predictions of several models trained with different initialization random seeds
* temporal activation regularization - add l2 regularization on the difference between the LSTM output activations at adjacent timesteps

You may notice that most of these suggestions are regularization techniques.  This dataset is considered fairly small, so regularization is one of the best ways to improve performance.

In the block below, describe the changes you made and how they affected performance.  If a change doesn't help, don't include it in the model you submit, but write about it here.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

***Exploring changes***

1. Increase embedding dimension from 128 to 512.

2. Add learning rate scheduler: progressively reduce learning rate by 1/4 times at some later epochs.

   *Both helps improve the performance.*

LSTM validation perplexity before your extra exploration: ***125.029217580714***

Final validation perplexity: ***109.53816410072704***

### Submission

Upload a submission with the following files to Gradescope:
* proj_1.ipynb (rename to match this exactly)
* lstm_predictions.npy (this should also include all improvements from your exploration)
* neural_trigram_predictions.npy
* trigram_kn_predictions.npy
* bigram_predictions.npy

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a completely different scale from your validation set perplexities due to truncating the distribution and selecting different text.  Don't worry if the values seem much worse.